GoogleNet, Inception 모듈

  1. 특징
     - CNN 계산 용량을 최적화하는 것을 고려
     - 전형적인 합성곱, 풀링 계층으로 시작하고, 이 정보는 9개의 인셉션 모듈 스택을 통과
     - 각 모듈에서 입력 특징 맵은 서로 다른 계층으로 구성도니 4개의 병렬 하위 블록에 전달되고, 이를 서로 다시 연결
     - 모든 합성곱과 풀링 계층은 padding='same', stride=1, activation='relu'
  2. 기여
     - 규모가 큰 블록과 병목을 보편화
     - 병목 계층으로 1x1 합성곱 계층을 사용
     - 완전 연결 계층 대신 풀링 계층 사용
     - 중간 소실로 경사 소실 문제 해결

In [1]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Conv2D, Flatten, MaxPool2D, Input

input_shape = (28, 28, 3)

model = Sequential()
model.add(Conv2D(32, kernel_size=(5, 5), input_shape=input_shape))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(10, activation='softmax'))

In [2]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 24, 24, 32)        2432      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 12, 12, 32)       0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 4608)              0         
                                                                 
 dense (Dense)               (None, 10)                46090     
                                                                 
Total params: 48,522
Trainable params: 48,522
Non-trainable params: 0
_________________________________________________________________


In [4]:
# 함수형 API 구현
def build_model():
    inputs = Input(shape=input_shape)
    conv1 = Conv2D(32, kernel_szie=(5,5))(inputs)
    maxpool1 = MaxPool2D(pool_size=(2, 2))(conv1)
    predictions = Dense(10, activation='relu')(Flatten()(maxpool1))
    
    model = Model(inputs=inputs, outputs=predictions)
    return model

   - 원시 버전의 인셉션 블록 생성

In [5]:
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, concatenate

def naive_inception_block(prev_layer, filters=[64, 128, 32]):
    conv1x1 = Conv2D(filters[0], kernel_size=(1, 1), padding='same', activation='relu')(prev_layer)
    conv3x3 = Conv2D(filters[1], kernel_size=(3, 3), padding='same', activation='relu')(prev_layer)
    conv5x5 = Conv2D(filters[2], kernel_size=(5, 5), padding='same', activation='relu')(prev_layer)
    max_pool = MaxPool2D((3, 3), strides=(1, 1), padding='same')(prev_layer)
    
    return concatenate([conv1x1, conv3x3, conv5x5, max_pool], axis=-1)
    

   - 케라스 모델에서 인셉션 모델

In [7]:
import tensorflow as tf
inception3_net = tf.keras.applications.InceptionV3(
    include_top=True, weights='imagenet', input_tensor=None, input_shape=None,
    pooling=None, classes=1000)

In [8]:
inception3_net.summary()

Model: "inception_v3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 299, 299, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_95 (Conv2D)             (None, 149, 149, 32  864         ['input_2[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization_94 (BatchN  (None, 149, 149, 32  96         ['conv2d_95[0][0]']              
 ormalization)                  )                                                      